In [1]:
from custompackage.load_data import *
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
# override SSL certificate verification failure
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# Google Colab compatibility
project_folder = ''

In [2]:
bs = 256
weighting = 'paired'
trials = 10
ds_set = ['emnist']
classes = 26

# Initialize for record keeping
# paired_test = np.zeros((len(ds_set), trials, classes, classes))
paired_test = np.load(project_folder+"results/confused_pairs_emnist.npy", allow_pickle=True)

# For each 10-class dataset
for k, ds in enumerate(ds_set):
    # Go through each class
    for t1 in range(10, 10 + classes):
        # and pair it with every other class
        for t2 in range(t1 + 1, 10 + classes):

            if paired_test[k, -1, t1-10, t2-10] != 0:
                continue

            # Load the binary classification dataloaders
            trainloaders, validloaders, testloader = dataset_weighted_split_all(bs, t1, t2, weighting, trials, ds)
            X_test = testloader.dataset.tensors[0]
            y_test = testloader.dataset.tensors[1]

            for m in range(trials):
                # Assign entirety of the datasets within each dataloader to a variable
                X_train = trainloaders[m].dataset.tensors[0]
                y_train = trainloaders[m].dataset.tensors[1]

                # initialize lda
                lda = LinearDiscriminantAnalysis()

                # fit to images, labels
                lda.fit(X_train, y_train)

                # see accuracy for validation set
                score_test = lda.score(X_test, y_test)

                print(ds, m, t1, t2, score_test)

                # record keeping

                paired_test[k, m, t1 - 10, t2 - 10] = score_test

                np.save(project_folder+'results/confused_pairs_emnist.npy', paired_test)

emnist 0 12 30 0.9473402833466987
emnist 1 12 30 0.9508152900294039
emnist 2 12 30 0.9497460572039561
emnist 3 12 30 0.9524191392675755
emnist 4 12 30 0.9518845228548516
emnist 5 12 30 0.9449345094894414
emnist 6 12 30 0.9542902967121091
emnist 7 12 30 0.9500133654103181
emnist 8 12 30 0.9438652766639936
emnist 9 12 30 0.9529537556802994
emnist 0 12 31 0.9380670611439842
emnist 1 12 31 0.9230769230769231
emnist 2 12 31 0.9226824457593689
emnist 3 12 31 0.9203155818540434
emnist 4 12 31 0.9270216962524654
emnist 5 12 31 0.9230769230769231
emnist 6 12 31 0.9246548323471401
emnist 7 12 31 0.9349112426035503
emnist 8 12 31 0.9136094674556213
emnist 9 12 31 0.9285996055226824
emnist 0 12 32 0.9387033398821218
emnist 1 12 32 0.9324165029469548
emnist 2 12 32 0.937524557956778
emnist 3 12 32 0.9335952848722986
emnist 4 12 32 0.9469548133595285
emnist 5 12 32 0.9449901768172888
emnist 6 12 32 0.9398821218074657
emnist 7 12 32 0.9461689587426326
emnist 8 12 32 0.9422396856581532
emnist 9 12 32 

KeyboardInterrupt: 